In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from IPython.display import clear_output          
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import tree
from collections import Counter
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from collections import Counter
from sklearn.neighbors import KNeighborsClassifier

## CALCULO DE HISTORIAL DE CANTIDAD DE POSTULACIONES A LAS DISTINTAS CATEGORIAS PARA UN MISMO USERID

- Leo todas las postulaciones

In [2]:
post_totales = pd.read_csv('post_totales.csv', usecols=['idaviso', 'idpostulante'])
post_totales.drop_duplicates(inplace=True)
post_totales.sort_values(by=['idpostulante'], ascending=True, inplace=True)
post_totales.reset_index(drop=True, inplace=True)
post_totales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6603752 entries, 0 to 6603751
Data columns (total 2 columns):
idaviso         int64
idpostulante    object
dtypes: int64(1), object(1)
memory usage: 100.8+ MB


In [3]:
post_totales.head()

,idaviso,idpostulante
0,1112384041,0z5Dmrd
1,1112420060,0z5Dmrd
2,1112428361,0z5JW1r
3,1112309728,0z5JW1r
4,1112292169,0z5JW1r


- Concateno todos los detalles de avisos

In [4]:
avisos_totales = pd.concat([pd.read_csv('datos_navent_fiuba/fiuba_6_avisos_detalle.csv', usecols=['idaviso', 'nombre_area']), pd.read_csv('fiuba_hasta_15_abril/fiuba_6_avisos_detalle.csv', usecols=['idaviso', 'nombre_area']), pd.read_csv('Fiuba desde 15 Abril/fiuba_6_avisos_detalle.csv', usecols=['idaviso', 'nombre_area']), pd.read_csv('fiuba_6_avisos_detalle_missing_nivel_laboral.csv', usecols=['idaviso', 'nombre_area'])])
avisos_totales.drop_duplicates(subset=['idaviso'], inplace=True)
avisos_totales.reset_index(drop=True, inplace=True)
avisos_totales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25288 entries, 0 to 25287
Data columns (total 2 columns):
idaviso        25288 non-null int64
nombre_area    25288 non-null object
dtypes: int64(1), object(1)
memory usage: 395.2+ KB


In [5]:
avisos_totales['nombre_area'].value_counts().size

188

In [6]:
#FUndamental usar para el label encoder una columna DF que contenga TODOS los posibles valores categoricos del nombre de area, es decir, lo saco del DF de TODOS los anuncios.
#ESto es para que si en un set no estan todos, igual los codifique de la misma forma. SI lo entreno con un set que no tiene todos, al ingresar un set con valores nuevos
#va a tirar error porque no tenia esos labels entrenados

#UNA VEZ ENTRENADO EL LABEL ENCODER, Y EL ONEHOTENCODER, USAR SIEMRE EL MISMO PARA EL RESTO DEL NOTEBOOK CODIFICAR LOS SETS DE DATA (TEST Y TRAIN) Y EL SET DE TEST100K

le = preprocessing.LabelEncoder()
avisos_totales['codigo_area'] = le.fit_transform(avisos_totales['nombre_area'].astype(str))

#PARA EL ONE HOT ENCODING, AVISARLE QUE LA CANTIDAD DE VALORES CATEGORICOS ES EL DE TODAS LAS CATEGORIAS EN EL DF DE TODOS LOS ANUNCIOS:188
enc = preprocessing.OneHotEncoder(188)   
enc.fit(avisos_totales['codigo_area'].values.reshape(-1,1))

avisos_totales.head()

,idaviso,nombre_area,codigo_area
0,8725750,Comercial,30
1,17903700,Salud,158
2,1000150677,Transporte,181
3,1000610287,Transporte,181
4,1000872556,Producción,143


- Merge con las postulaciones, y borro rows que para un mismo postulante sean de la misma area

In [7]:
post_totales = post_totales.merge(avisos_totales[['idaviso','codigo_area']], how='inner', on='idaviso')

#Borro los duplicados de idpostulante y codigo_area, quiero solo ver si se postulo al menos en una ocasion a esa categoria, no cuantas veces lo hizo
#post_totales.drop_duplicates(subset=['idpostulante', 'codigo_area'], inplace=True)

post_totales['codigo_area'] = post_totales['codigo_area'].astype('int16')

#Quiero ver cuantas veces se postulo a cada categoria, agrupo y saco la cantidad por cada area
post_totales = post_totales.groupby(['idpostulante','codigo_area']).count()
post_totales.rename(columns={'idaviso':'count'}, inplace=True)

post_totales.head(10)

count
idpostulante codigo_area       
0z5Dmrd      154              2
0z5JW1r      5                1
             30               1
             51               1
             82               1
             83               2
             185              1
0z5VvGv      2                1
             14              11
             26               4

In [8]:
post_totales.reset_index(drop=False, inplace=True)
post_totales.head(5)

,idpostulante,codigo_area,count
0,0z5Dmrd,154,2
1,0z5JW1r,5,1
2,0z5JW1r,30,1
3,0z5JW1r,51,1
4,0z5JW1r,82,1


In [9]:
post_totales.sort_values(by='idpostulante', ascending=True, inplace=True)
post_totales.set_index('idpostulante', drop=False, inplace=True)
post_totales.head()

,idpostulante,codigo_area,count
idpostulante,,,
0z5Dmrd,0z5Dmrd,154,2
0z5JW1r,0z5JW1r,5,1
0z5JW1r,0z5JW1r,30,1
0z5JW1r,0z5JW1r,51,1
0z5JW1r,0z5JW1r,82,1
0z5JW1r,0z5JW1r,83,2
0z5JW1r,0z5JW1r,185,1
0z5VvGv,0z5VvGv,185,13
0z5VvGv,0z5VvGv,182,3


- Cargo Dataset para operar

In [10]:
data_set_complete = pd.read_csv('data_set_complete.csv', parse_dates=['fechapostulacion', 'ultima_vista'], dtype={'sepostulo':'int8', 'idaviso':'int64', 'idpostulante':'object', 'veces_visto':'int16', 'estudios':'category', 'estado':'category', 'edad':'int8', 'sexo':'category','nombre_zona':'category', 'tipo_de_trabajo':'category', 'nivel_laboral':'category', 'nombre_area':'category'})
data_set_complete.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13208286 entries, 0 to 13208285
Data columns (total 16 columns):
sepostulo               int8
idaviso                 int64
idpostulante            object
fechapostulacion        datetime64[ns]
ultima_vista            datetime64[ns]
veces_visto             int16
estudios                category
estado                  category
sexo                    category
edad                    int8
titulo                  object
nombre_zona             category
tipo_de_trabajo         category
nivel_laboral           category
nombre_area             category
denominacion_empresa    object
dtypes: category(7), datetime64[ns](2), int16(1), int64(1), int8(2), object(3)
memory usage: 755.8+ MB


In [11]:
#FIltro aquellos con campos nulos o sexo no declara, tengo muchas muestras
data_set_complete.dropna(subset=['estudios','estado','sexo','edad','nombre_zona','tipo_de_trabajo','nivel_laboral','nombre_area'], inplace=True)
data_set_complete = data_set_complete[data_set_complete['sexo']!='NO_DECLARA']

#Achico a 1millon para maniabrabilidad en terminos de performance
data = data_set_complete.sample(1000000)

del data_set_complete

In [12]:
#Dropeo duplicados asi tengo una lista de todos los users ids distintos en el set de 1millon
data_users_ids = data['idpostulante'].to_frame().drop_duplicates().reset_index(drop=True)
data_users_ids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 352209 entries, 0 to 352208
Data columns (total 1 columns):
idpostulante    352209 non-null object
dtypes: object(1)
memory usage: 2.7+ MB


In [13]:
#lo voy a usar para guardar el encoding, mismo indice que a data_users_ids para tener asociado cada row a la misma idpostulante
data_users_encoding = data['idpostulante'].to_frame().drop_duplicates().reset_index(drop=True)

#Auxiliar
data_users_encoding['aux'] = 0

#Encodeo el valor categorico 0 para crear una matriz de encoding vacia
encoded_areas = enc.transform(data_users_encoding['aux'].values.reshape(-1,1))
encoded_areas_df = pd.DataFrame(encoded_areas.toarray()).astype('int8')

data_users_encoding = pd.concat([data_users_encoding, encoded_areas_df], axis=1)

del encoded_areas, encoded_areas_df

#Borro las demas columnas para que me queden solo las 188 de encoding. A la primer columna de encoding la vuelvo a setear en 0 ya que al encodear antes el valor
#categorico 0, esa columna esta completa con 1
data_users_encoding.drop(columns=['idpostulante','aux'], inplace=True)
data_users_encoding[0] = 0

data_users_encoding.head()

,0,1,2,3,4,5,6,7,8,9,...,178,179,180,181,182,183,184,185,186,187
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


- Extraccion del historial de veces que se postulo a cada categoria

In [44]:
progress = 0
aux = 0
progress_size = data_users_ids.size

while(progress < progress_size):
    
    try:
        aux = post_totales.loc[data_users_ids.iat[progress,0],['codigo_area','count']]
        
        if(aux.size!=2):
            for index, row in aux.iterrows():
                data_users_encoding.iat[progress, np.int(row[0])] = np.int(row[1])
        else:
            data_users_encoding.iat[progress, np.int(aux[0])] = np.int(aux[1])  
            
    except KeyError:
        pass
    
    progress += 1
    clear_output(wait=True)
    print(progress, 'of', progress_size)

352209 of 352209


In [45]:
data_users = pd.concat([data_users_ids, data_users_encoding], axis=1)
data_users.head()

,idpostulante,0,1,2,3,4,5,6,7,8,...,178,179,180,181,182,183,184,185,186,187
0,KBrGMEA,0,0,7,0,4,3,0,0,0,...,0,0,0,0,0,0,0,41,2,0
1,lD5qoWz,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6qZAA4,0,0,44,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,2,0
3,YjlXLBZ,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,5,0,0
4,Erx039,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
data_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 352209 entries, 0 to 352208
Columns: 189 entries, idpostulante to 187
dtypes: int64(1), int8(187), object(1)
memory usage: 68.2+ MB


## Ahora la parte de operacion y clasificacion

### Concateno toda la info de educacion

In [47]:
edu_totales = pd.concat([pd.read_csv('datos_navent_fiuba/fiuba_1_postulantes_educacion.csv'), pd.read_csv('fiuba_hasta_15_abril/fiuba_1_postulantes_educacion.csv'), pd.read_csv('Fiuba desde 15 Abril/fiuba_1_postulantes_educacion.csv')])
edu_totales.drop_duplicates(inplace=True)

edu_totales = edu_totales.rename(columns={'nombre':'estudios'})
edu_totales['estudios'] = edu_totales['estudios'].astype('category')
edu_totales['estado'] = edu_totales['estado'].astype('category')

#Saco abandonados y me separo graduados de en curso

edu_graduados = edu_totales[edu_totales['estado']=='Graduado'].drop(columns='estado')
edu_encurso = edu_totales[edu_totales['estado']=='En Curso'].drop(columns='estado')

del edu_totales

# Creo nuevas columnas indicando el orden del nivel del estudio

edu_estudios_list = ['Ninguno','Secundario', 'Otro', 'Terciario/Técnico', 'Universitario', 'Posgrado', 'Master', 'Doctorado']

edu_graduados['estudio_graduado'] = edu_graduados['estudios'].apply(lambda x: edu_estudios_list.index(x)).astype('int8')
edu_graduados.sort_values('estudio_graduado', ascending=True, inplace=True)
edu_graduados.drop_duplicates(subset=['idpostulante'], keep='last', inplace=True)
edu_graduados.sort_values(by=['idpostulante'], ascending=True, inplace=True)
edu_graduados.rename(columns={'estudios':'graduado'}, inplace=True)
edu_graduados.reset_index(drop=True, inplace=True)

edu_encurso['estudio_encurso'] = edu_encurso['estudios'].apply(lambda x: edu_estudios_list.index(x)).astype('int8')
edu_encurso.sort_values('estudio_encurso', ascending=True, inplace=True)
edu_encurso.drop_duplicates(subset=['idpostulante'], keep='last', inplace=True)
edu_encurso.sort_values(by=['idpostulante'], ascending=True, inplace=True)
edu_encurso.rename(columns={'estudios':'encurso'}, inplace=True)
edu_encurso.reset_index(drop=True, inplace=True)

edu_totales = pd.merge(edu_encurso, edu_graduados, how='outer', on='idpostulante')

del edu_graduados, edu_encurso

edu_totales.head()

,idpostulante,encurso,estudio_encurso,graduado,estudio_graduado
0,0zB028M,Terciario/Técnico,3.0,Otro,2.0
1,0zB03O9,Terciario/Técnico,3.0,Secundario,1.0
2,0zB0PLa,Universitario,4.0,Secundario,1.0
3,0zB0WZE,Universitario,4.0,NaN,NaN
4,0zB0Xwd,Universitario,4.0,Secundario,1.0


### Concateno todos los set de vistas y calculo cantidad de vistas de cada usuario a cada anuncio

In [48]:
vistas_totales = pd.concat([pd.read_csv('datos_navent_fiuba/fiuba_3_vistas.csv'), pd.read_csv('fiuba_hasta_15_abril/fiuba_3_vistas.csv'), pd.read_csv('Fiuba desde 15 Abril/fiuba_3_vistas.csv')])

def normalize_str(date_str):
    return (date_str[:10]+' '+date_str[11:19])
    
vistas_totales['timestamp'] = vistas_totales['timestamp'].apply(normalize_str)
vistas_totales['timestamp'] = pd.to_datetime(vistas_totales['timestamp'])

vistas_totales.drop_duplicates(inplace=True)

vistas_totales.rename(columns={'idAviso':'idaviso'}, inplace=True)

#Calculo cantidad de vistas de cada usuario a cada anuncio

vistas_totales_count = vistas_totales.groupby(by=['idaviso', 'idpostulante']).count()
vistas_totales_count.reset_index(drop=False, inplace=True)
vistas_totales_count.rename(columns={'timestamp':'veces_visto'}, inplace=True)

del vistas_totales

vistas_totales_count.head()

,idaviso,idpostulante,veces_visto
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


### Trabajo sobre la data, realizo one hot encoding manual

In [49]:
data.drop(columns=['estudios', 'estado'], inplace=True)
data = data.merge(edu_totales[['idpostulante','estudio_encurso','estudio_graduado']], on='idpostulante', how='left')

data['estudio_encurso'] = data['estudio_encurso'].fillna(value=0).astype('int8')
data['estudio_graduado'] = data['estudio_graduado'].fillna(value=0).astype('int8')

In [50]:
data['tipo_de_trabajo'].value_counts()

Full-time          847898
Part-time          136530
Pasantia             4859
Por Horas            3523
Temporario           2686
Fines de Semana      1525
Por Contrato         1484
Teletrabajo          1087
Primer empleo         356
Voluntario             52
Name: tipo_de_trabajo, dtype: int64

In [51]:
data['tipo_full-time'] = data['tipo_de_trabajo'].apply(lambda x: 1 if (x=='Full-time') else 0).astype('int8')
data['tipo_part-time'] = data['tipo_de_trabajo'].apply(lambda x: 1 if (x=='Part-time') else 0).astype('int8')
data['tipo_otro'] = data['tipo_de_trabajo'].apply(lambda x: 1 if ((x != 'Full-time') & (x != 'Part-time')) else 0).astype('int8')

data.drop(columns='tipo_de_trabajo', inplace=True)

In [52]:
data['zona_gba'] = data['nombre_zona'].apply(lambda x: 1 if ((x=='Gran Buenos Aires') or (x=='GBA Oeste')) else 0).astype('int8')
data['zona_caba'] = data['nombre_zona'].apply(lambda x: 1 if (x=='Capital Federal') else 0).astype('int8')

data.drop(columns='nombre_zona', inplace=True)

In [53]:
data['nivel_senior'] = data['nivel_laboral'].apply(lambda x: 1 if (x=='Senior / Semi-Senior') else 0).astype('int8')
data['nivel_junior'] = data['nivel_laboral'].apply(lambda x: 1 if (x=='Junior') else 0).astype('int8')
data['nivel_otro'] = data['nivel_laboral'].apply(lambda x: 1 if (x=='Otro') else 0).astype('int8')
data['nivel_jefe'] = data['nivel_laboral'].apply(lambda x: 1 if (x=='Jefe / Supervisor / Responsable') else 0).astype('int8')
data['nivel_gerente'] = data['nivel_laboral'].apply(lambda x: 1 if (x=='Gerencia / Alta Gerencia / Dirección') else 0).astype('int8')

data.drop(columns='nivel_laboral', inplace=True)

In [54]:
data['sexo'] = data['sexo'].apply(lambda x: 1 if (x=='MASC') else 0).astype('int8')
data.reset_index(drop=True, inplace=True)
data.head()

,sepostulo,idaviso,idpostulante,fechapostulacion,ultima_vista,veces_visto,sexo,edad,titulo,nombre_area,...,tipo_full-time,tipo_part-time,tipo_otro,zona_gba,zona_caba,nivel_senior,nivel_junior,nivel_otro,nivel_jefe,nivel_gerente
0,1,1112241774,KBrGMEA,2018-03-07 18:51:56,NaT,1,1,19,Cajeros/Repositores Eventuales - GBA Zona Oeste,Comercial,...,1,0,0,1,0,1,0,0,0,0
1,0,1112165512,lD5qoWz,NaT,NaT,0,1,20,Jefe de Salud y Seguridad Ocupacional,Ingeniería Química,...,1,0,0,1,0,0,0,0,1,0
2,1,1112339084,6qZAA4,2018-02-21 13:09:26,NaT,1,0,31,Administrativo - Plan Sueldo (Gerencia de Oper...,Administración,...,1,0,0,1,0,0,1,0,0,0
3,1,1112278128,YjlXLBZ,2018-01-22 21:14:15,NaT,1,1,25,GESTORES DE COBRANZAS - PART TIME,Oficios y Profesiones,...,0,1,0,1,0,1,0,0,0,0
4,0,1112337991,Erx039,NaT,NaT,0,0,27,CAJERAS PART TIME,Tesorería,...,0,1,0,1,0,0,0,1,0,0


In [55]:
# TIENE QUE SER EL LABEL ENCODER DE ANTES, EL BASADO EN EL TOTAL DE AVISOS, QUE TIENE TODAS LAS CATEGORIAS
data['nombre_area'] = le.transform(data['nombre_area'].astype(str))

### Con veces_visto (en inexistentes usamos default en 0 para todos)

In [56]:
data['veces_visto'] = (pd.merge(data[['idaviso','idpostulante']], vistas_totales_count, on=['idaviso', 'idpostulante'], how='left'))['veces_visto']
data['veces_visto'].fillna(value=0, inplace=True)
data['veces_visto'] = data['veces_visto'].astype('int16')

veces_visto_fillna_0 = data['veces_visto'] #Lo guardo para futuras comparaciones

In [57]:
# Aca meto la info del historial (matriz de 188 con el one hot encoding)
data = data.merge(data_users, how='left', on='idpostulante')
data.head()

,sepostulo,idaviso,idpostulante,fechapostulacion,ultima_vista,veces_visto,sexo,edad,titulo,nombre_area,...,178,179,180,181,182,183,184,185,186,187
0,1,1112241774,KBrGMEA,2018-03-07 18:51:56,NaT,0,1,19,Cajeros/Repositores Eventuales - GBA Zona Oeste,30,...,0,0,0,0,0,0,0,41,2,0
1,0,1112165512,lD5qoWz,NaT,NaT,0,1,20,Jefe de Salud y Seguridad Ocupacional,99,...,0,0,0,0,0,0,0,0,0,0
2,1,1112339084,6qZAA4,2018-02-21 13:09:26,NaT,0,0,31,Administrativo - Plan Sueldo (Gerencia de Oper...,2,...,0,0,0,0,0,0,0,1,2,0
3,1,1112278128,YjlXLBZ,2018-01-22 21:14:15,NaT,0,1,25,GESTORES DE COBRANZAS - PART TIME,131,...,0,0,0,0,0,0,0,5,0,0
4,0,1112337991,Erx039,NaT,NaT,0,0,27,CAJERAS PART TIME,175,...,0,0,0,0,0,0,0,0,0,0


### Pruebo con Random FOrest

In [58]:
#Saco edad porque vimos que resta score
train_set, test_set, train_classes, test_classes = train_test_split(data.drop(columns=['edad','sepostulo','idaviso','idpostulante', 'fechapostulacion','ultima_vista','titulo','denominacion_empresa']), data['sepostulo'], test_size=0.3)

In [61]:
#sin edad, con vecesvisto
clf1 = RandomForestClassifier(n_estimators=20).fit(train_set, train_classes)
print('Test:', clf1.score(test_set, test_classes), 'Entrenamiento:', clf1.score(train_set, train_classes))

Test: 0.8568966666666666 Entrenamiento: 0.9928285714285714


- El score del historial unicamente binario (1 o 0, se postulo o no a esa categoria) habia dado:
    - Test: 0.85448 Entrenamiento: 0.9893614285714286

### Pruebo GRidSearch con variacion de parametros para intentar reducir el overfitting del random forest

In [64]:
parameters = {'min_samples_leaf':[1, 10, 100, 1000, 10000, 100000]}

randtree = RandomForestClassifier(n_jobs=-1)
clf2 = GridSearchCV(randtree, parameters)
clf2.fit(train_set, train_classes)

print('Test:', clf2.score(test_set, test_classes), 'Entrenamiento:', clf2.score(train_set, train_classes))

Test: 0.85705 Entrenamiento: 0.8739871428571429


In [65]:
clf2.best_params_

{'min_samples_leaf': 10}

In [70]:
parameters = {'max_depth':[1, 10, 100, 1000, 10000, 100000]}

randtree = RandomForestClassifier(n_jobs=-1, min_samples_leaf=10)
clf3 = GridSearchCV(randtree, parameters)
clf3.fit(train_set, train_classes)

print('Test:', clf3.score(test_set, test_classes), 'Entrenamiento:', clf3.score(train_set, train_classes))

Test: 0.85575 Entrenamiento: 0.8711857142857143


In [71]:
clf3.best_params_

{'max_depth': 100000}

In [72]:
parameters = {'min_samples_split':[2, 20, 200, 2000, 20000, 200000]}

randtree = RandomForestClassifier(n_jobs=-1, min_samples_leaf=10)
clf4 = GridSearchCV(randtree, parameters)
clf4.fit(train_set, train_classes)

print('Test:', clf4.score(test_set, test_classes), 'Entrenamiento:', clf4.score(train_set, train_classes))

Test: 0.85433 Entrenamiento: 0.8711871428571428


In [73]:
clf4.best_params_

{'min_samples_split': 2}

In [75]:
parameters = {'min_samples_leaf':[7, 8, 9, 10, 11, 12, 13], 'min_samples_split':[2,3,4,5]}

randtree = RandomForestClassifier(n_jobs=-1)
clf5 = GridSearchCV(randtree, parameters)
clf5.fit(train_set, train_classes)

print('Test:', clf5.score(test_set, test_classes), 'Entrenamiento:', clf5.score(train_set, train_classes))

Test: 0.8579866666666667 Entrenamiento: 0.8806528571428571


In [76]:
clf5.best_params_

{'min_samples_leaf': 7, 'min_samples_split': 4}

In [82]:
#Entonces, probamos con esos parametros
clf6 = RandomForestClassifier(n_estimators=20, min_samples_leaf=7, min_samples_split=4).fit(train_set, train_classes)
print('Test:', clf6.score(test_set, test_classes), 'Entrenamiento:', clf6.score(train_set, train_classes))

Test: 0.85997 Entrenamiento: 0.8829385714285715


## Probamos GradientBoosting

In [77]:
clf7 = GradientBoostingClassifier()
clf7.fit(train_set, train_classes)

print('Test:', clf7.score(test_set, test_classes), 'Entrenamiento:', clf7.score(train_set, train_classes))

Test: 0.83367 Entrenamiento: 0.8345742857142857


## Probamos AdaptativeBoosting

In [83]:
clf8 = AdaBoostClassifier().fit(train_set, train_classes)

print('Test:', clf8.score(test_set, test_classes), 'Entrenamiento:', clf8.score(train_set, train_classes))

Test: 0.80519 Entrenamiento: 0.8056842857142857


### Cargo un test100k para hacer una prueba

In [84]:
test100k = pd.read_csv('test_final_100k_complete.csv', parse_dates=['fechapostulacion', 'ultima_vista'], dtype={'sepostulo':'int8', 'idaviso':'int64', 'idpostulante':'object', 'veces_visto':'int16', 'estudios':'category', 'estado':'category', 'edad':'int8', 'sexo':'category','nombre_zona':'category', 'tipo_de_trabajo':'category', 'nivel_laboral':'category', 'nombre_area':'category'})

aux = ['MASC', 'FEM']
test100k['sexo'] = test100k['sexo'].apply(lambda x: aux[np.random.randint(2)] if (x == 'NO_DECLARA') else x)
del aux

test100k['nombre_zona'].fillna(value='Gran Buenos Aires',inplace=True)
test100k['tipo_de_trabajo'].fillna(value='Full-time',inplace=True)
test100k['nivel_laboral'].fillna(value='Senior / Semi-Senior',inplace=True)
test100k['nombre_area'].fillna(value='Ventas',inplace=True)

In [85]:
test100k.drop(columns=['estudios', 'estado'], inplace=True)
test100k = test100k.merge(edu_totales[['idpostulante','estudio_encurso','estudio_graduado']], on='idpostulante', how='left')

test100k['estudio_encurso'] = test100k['estudio_encurso'].fillna(value=0).astype('int8')
test100k['estudio_graduado'] = test100k['estudio_graduado'].fillna(value=0).astype('int8')

In [86]:
test100k['tipo_full-time'] = test100k['tipo_de_trabajo'].apply(lambda x: 1 if (x=='Full-time') else 0).astype('int8')
test100k['tipo_part-time'] = test100k['tipo_de_trabajo'].apply(lambda x: 1 if (x=='Part-time') else 0).astype('int8')
test100k['tipo_otro'] = test100k['tipo_de_trabajo'].apply(lambda x: 1 if ((x != 'Full-time') & (x != 'Part-time')) else 0).astype('int8')

test100k.drop(columns='tipo_de_trabajo', inplace=True)

In [87]:
test100k['zona_gba'] = test100k['nombre_zona'].apply(lambda x: 1 if ((x=='Gran Buenos Aires') or (x=='GBA Oeste')) else 0).astype('int8')
test100k['zona_caba'] = test100k['nombre_zona'].apply(lambda x: 1 if (x=='Capital Federal') else 0).astype('int8')

test100k.drop(columns='nombre_zona', inplace=True)

In [88]:
test100k['nivel_senior'] = test100k['nivel_laboral'].apply(lambda x: 1 if (x=='Senior / Semi-Senior') else 0).astype('int8')
test100k['nivel_junior'] = test100k['nivel_laboral'].apply(lambda x: 1 if (x=='Junior') else 0).astype('int8')
test100k['nivel_otro'] = test100k['nivel_laboral'].apply(lambda x: 1 if (x=='Otro') else 0).astype('int8')
test100k['nivel_jefe'] = test100k['nivel_laboral'].apply(lambda x: 1 if (x=='Jefe / Supervisor / Responsable') else 0).astype('int8')
test100k['nivel_gerente'] = test100k['nivel_laboral'].apply(lambda x: 1 if (x=='Gerencia / Alta Gerencia / Dirección') else 0).astype('int8')

test100k.drop(columns='nivel_laboral', inplace=True)

In [89]:
test100k['sexo'] = test100k['sexo'].apply(lambda x: 1 if (x=='MASC') else 0).astype('int8')
test100k.reset_index(drop=True, inplace=True)
test100k.head()

,id,idaviso,idpostulante,fechapostulacion,ultima_vista,veces_visto,sexo,edad,titulo,nombre_area,...,tipo_full-time,tipo_part-time,tipo_otro,zona_gba,zona_caba,nivel_senior,nivel_junior,nivel_otro,nivel_jefe,nivel_gerente
0,0,739260,6M9ZQR,NaT,NaT,0,0,42,Asistente Comercial,Comercial,...,1,0,0,0,1,0,0,0,1,0
1,1,739260,6v1xdL,NaT,NaT,0,1,30,Asistente Comercial,Comercial,...,1,0,0,0,1,0,0,0,1,0
2,2,739260,ezRKm9,NaT,NaT,0,0,36,Asistente Comercial,Comercial,...,1,0,0,0,1,0,0,0,1,0
3,3,758580,1Q35ej,NaT,NaT,0,1,68,Oracle Financials Sr. Application Developer,Tecnologia / Sistemas,...,1,0,0,0,1,0,0,1,0,0
4,4,758580,EAN4J6,NaT,NaT,0,0,32,Oracle Financials Sr. Application Developer,Tecnologia / Sistemas,...,1,0,0,0,1,0,0,1,0,0


In [90]:
test100k['nombre_area'] = le.transform(test100k['nombre_area'].astype(str))

In [91]:
test100k = test100k.merge(data_users, how='left', on='idpostulante')
test100k.head()

,id,idaviso,idpostulante,fechapostulacion,ultima_vista,veces_visto,sexo,edad,titulo,nombre_area,...,178,179,180,181,182,183,184,185,186,187
0,0,739260,6M9ZQR,NaT,NaT,0,0,42,Asistente Comercial,30,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,739260,6v1xdL,NaT,NaT,0,1,30,Asistente Comercial,30,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0
2,2,739260,ezRKm9,NaT,NaT,0,0,36,Asistente Comercial,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,758580,1Q35ej,NaT,NaT,0,1,68,Oracle Financials Sr. Application Developer,170,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,758580,EAN4J6,NaT,NaT,0,0,32,Oracle Financials Sr. Application Developer,170,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
test100k.fillna(value=0.0, inplace=True)
test100k.head()

,id,idaviso,idpostulante,fechapostulacion,ultima_vista,veces_visto,sexo,edad,titulo,nombre_area,...,178,179,180,181,182,183,184,185,186,187
0,0,739260,6M9ZQR,0.0,0,0,0,42,Asistente Comercial,30,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,739260,6v1xdL,0.0,0,0,1,30,Asistente Comercial,30,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0
2,2,739260,ezRKm9,0.0,0,0,0,36,Asistente Comercial,30,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,758580,1Q35ej,0.0,0,0,1,68,Oracle Financials Sr. Application Developer,170,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,758580,EAN4J6,0.0,0,0,0,32,Oracle Financials Sr. Application Developer,170,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [95]:
test100k['sepostulo'] = clf6.predict(test100k.drop(columns=['id', 'edad', 'idaviso','idpostulante', 'fechapostulacion','ultima_vista','titulo', 'denominacion_empresa']).astype(int))

In [96]:
test100k[['id','sepostulo']].to_csv('Resultados/randforest_gridsearch_sinonehotareas_conhistorialonehot.csv', index=False)